In [7]:
import json
import query_elasticsearch as es

# This is the gold training data from pilot 3 or 4
evidence_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot3_twowingos/110718-training-data/evidence.json"
claim_persp_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot3_twowingos/110718-training-data/claim_perspective.json"
annotation_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot3_twowingos/110718-training-data/gold_annotation.json"

with open(evidence_path, 'r', encoding='utf-8') as fin:
    evidences = json.load(fin)

with open(claim_persp_path, 'r', encoding='utf-8') as fin:
    persps = json.load(fin)
    
with open(annotation_path, 'r', encoding='utf-8') as fin:
    annotations = json.load(fin)

print(len(evidences))
print(len(persps))
print(len(annotations))

8122
4645
4645


In [8]:
import pandas as pd

adf = pd.DataFrame(annotations)
adf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4645 entries, 0 to 4644
Data columns (total 2 columns):
evidence_id       4645 non-null int64
perspective_id    4645 non-null int64
dtypes: int64(2)
memory usage: 72.7 KB


In [ ]:
# For each perspective generate evidence candidates
results = []
for p in persps:
    # top 50 candidates
    _cands = es.get_top_evidences(p['title'])
    cands = [(eid, score) for e, eid, score in _cands]
    results.append({
        'persp_id': p['id'],
        'candidates': cands
    })

# save results
out_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot6_lucene/pp_2_ev.json"

with open(out_path, 'w') as fout:
    json.dump(results, fout)


In [17]:
# For each perspective, measure top k accuracy of evidence candidates
def get_gold_evidence(perspective_id, adf):
    q = adf.loc[adf.perspective_id == perspective_id]
    if len(q) > 0:
        return q.iloc[0].evidence_id
    else:
        return None
    
k = [1, 10, 25, 50]
correct = [0 for _k in k]

for res in results:
    gold = get_gold_evidence(res['persp_id'], adf)
    for idx, _k in enumerate(k):
        cands = [eid for eid, score in res['candidates']]
        
        if gold in cands[:_k]:
            correct[idx] += 1

total = len(results)


for idx, _k in enumerate(k):
    print("Top {} accuracy: {}".format(_k, correct[idx] / total))

Top 1 accuracy: 0.36404736275565125
Top 10 accuracy: 0.7791173304628632
Top 25 accuracy: 0.8716899892357374
Top 50 accuracy: 0.9121636167922498


In [27]:
# The other way around: for each evidence, generate perspective candidates
from elasticsearch.exceptions import RequestError

results = []
for e in evidences:
    # top 50 candidates
    try:
        _cands = es.get_top_perspectives(e['content'])
    except RequestError:
        _cands = []
    cands = [(pid, score) for p, pid, score in _cands]
    results.append({
        'evidence_id': e['id'],
        'candidates': cands
    })

# save results
out_path = "/home/squirrel/ccg-new/projects/perspective/data/pilot6_lucene/ev_2_pp.json"

with open(out_path, 'w') as fout:
    json.dump(results, fout)


4544 documents found:
4591 documents found:
4630 documents found:
4602 documents found:
4625 documents found:
4574 documents found:
4579 documents found:
4622 documents found:
4584 documents found:
4614 documents found:
4580 documents found:
4608 documents found:
4486 documents found:
4608 documents found:
4588 documents found:
4604 documents found:
4489 documents found:
4610 documents found:
4621 documents found:
4586 documents found:
4618 documents found:
4630 documents found:
4610 documents found:
4609 documents found:
4572 documents found:
4625 documents found:
4551 documents found:
4569 documents found:
4635 documents found:
4622 documents found:
4634 documents found:
4625 documents found:
4548 documents found:
4604 documents found:
4625 documents found:
4564 documents found:
4575 documents found:
4622 documents found:
4621 documents found:
4514 documents found:
4631 documents found:
4544 documents found:
4634 documents found:
4633 documents found:
4623 documents found:
4588 docum

4614 documents found:
4623 documents found:
4584 documents found:
4598 documents found:
4591 documents found:
4523 documents found:
4608 documents found:
4415 documents found:
4590 documents found:
4300 documents found:
4515 documents found:
4611 documents found:
4594 documents found:
4625 documents found:
4574 documents found:
4602 documents found:
4629 documents found:
4605 documents found:
4610 documents found:
4595 documents found:
4634 documents found:
4619 documents found:
4626 documents found:
4633 documents found:
4622 documents found:
4633 documents found:
4606 documents found:
4602 documents found:
4549 documents found:
4568 documents found:
4613 documents found:
4584 documents found:
4616 documents found:
4475 documents found:
4625 documents found:
4626 documents found:
4533 documents found:
4428 documents found:
4606 documents found:
4553 documents found:
4606 documents found:
4320 documents found:
4576 documents found:
4613 documents found:
4621 documents found:
4621 docum

4608 documents found:
4608 documents found:
4580 documents found:
4577 documents found:
4605 documents found:
4523 documents found:
4604 documents found:
4608 documents found:
4514 documents found:
4616 documents found:
4486 documents found:
4626 documents found:
4526 documents found:
4577 documents found:
4630 documents found:
4619 documents found:
4634 documents found:
4641 documents found:
4636 documents found:
4629 documents found:
4641 documents found:
4636 documents found:


GET http://bronte.cs.illinois.edu:8080/perspectivesandclaims/text/_search?size=50 [status:400 request:0.069s]


4638 documents found:
4636 documents found:
4633 documents found:
4635 documents found:
4630 documents found:
4635 documents found:
4605 documents found:
4582 documents found:
4602 documents found:
4620 documents found:
4599 documents found:
4572 documents found:
4625 documents found:
4623 documents found:
4606 documents found:
4625 documents found:
4625 documents found:
4615 documents found:
4628 documents found:
4620 documents found:
4620 documents found:
4611 documents found:
4595 documents found:
4608 documents found:
4546 documents found:
4525 documents found:
4576 documents found:
4631 documents found:
4526 documents found:
4577 documents found:
4586 documents found:
4567 documents found:
4604 documents found:
4580 documents found:
4543 documents found:
4596 documents found:
4564 documents found:
4370 documents found:
4551 documents found:
4515 documents found:
4559 documents found:
4535 documents found:
4614 documents found:
4611 documents found:
4589 documents found:
4589 docum

...

4625 documents found:
4635 documents found:
4635 documents found:
4616 documents found:
4635 documents found:
4623 documents found:
4635 documents found:
4633 documents found:
4621 documents found:
4634 documents found:
4630 documents found:
4527 documents found:
4633 documents found:
4622 documents found:
4633 documents found:
4634 documents found:
4624 documents found:
4633 documents found:
4636 documents found:
4633 documents found:
4621 documents found:
4635 documents found:
4632 documents found:
4634 documents found:
4631 documents found:
4635 documents found:
4626 documents found:
4624 documents found:
4623 documents found:
4619 documents found:
4455 documents found:
4418 documents found:
4632 documents found:
4615 documents found:
4629 documents found:
4501 documents found:
4614 documents found:
4589 documents found:
4629 documents found:
4601 documents found:
4636 documents found:
4633 documents found:
4638 documents found:
4626 documents found:
4634 documents found:
4632 docum

GET http://bronte.cs.illinois.edu:8080/perspectivesandclaims/text/_search?size=50 [status:400 request:0.061s]
GET http://bronte.cs.illinois.edu:8080/perspectivesandclaims/text/_search?size=50 [status:400 request:0.055s]


4641 documents found:
4640 documents found:
4634 documents found:
4340 documents found:
4605 documents found:
4560 documents found:
4614 documents found:
4565 documents found:
4537 documents found:
4575 documents found:
4572 documents found:
4635 documents found:
4623 documents found:
4617 documents found:
4634 documents found:
4625 documents found:
4584 documents found:
4537 documents found:
4618 documents found:
4618 documents found:


GET http://bronte.cs.illinois.edu:8080/perspectivesandclaims/text/_search?size=50 [status:400 request:0.060s]


4611 documents found:
4593 documents found:
4607 documents found:
4620 documents found:
4530 documents found:
4616 documents found:
4631 documents found:
4543 documents found:
4557 documents found:
4614 documents found:
4601 documents found:
4540 documents found:
4572 documents found:
4608 documents found:
4631 documents found:
4520 documents found:
4562 documents found:
4066 documents found:
4429 documents found:
4507 documents found:
4508 documents found:
4572 documents found:
4021 documents found:
4578 documents found:
4617 documents found:
4109 documents found:
4318 documents found:
4322 documents found:
4272 documents found:
4608 documents found:
4632 documents found:
4584 documents found:
4627 documents found:


In [31]:
# For each evidence, measure top k accuracy of perspective candidates
def get_gold_perspective(evidence_id, adf):
    q = adf.loc[adf.evidence_id == evidence_id]
    if len(q) > 0:
        return q.iloc[0].perspective_id
    else:
        return None
    
k = [1, 10, 25, 50]
correct = [0 for _k in k]

for res in results:
    gold = get_gold_perspective(res['evidence_id'], adf)
    for idx, _k in enumerate(k):
        if len(res['candidates']) > 0:
            cands = [pid for pid, score in res['candidates']]
            if gold in cands[:_k]:
                correct[idx] += 1

total = len(results)


for idx, _k in enumerate(k):
    print("Top {} accuracy: {}".format(_k, correct[idx] / total))

Top 1 accuracy: 0.2187884757448904
Top 10 accuracy: 0.4054420093573012
Top 25 accuracy: 0.4509972913075597
Top 50 accuracy: 0.4758680128047279
